In [18]:
import pandas as pd
import seaborn as sns
import numpy as np
import linearmodels as lm
import matplotlib
import math
import statsmodels.formula.api as smf

from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects
from linearmodels.panel import FirstDifferenceOLS
from linearmodels.panel import compare
from matplotlib import pyplot as plt

from pandas.api.types import is_numeric_dtype

pd.set_option('display.max_columns', 500)

%matplotlib inline

In [19]:
# read the main data set
df=pd.read_csv('final_dataset_oct_24.csv',
                    encoding='utf-8')


In [20]:
# exclude if ridership is zero--missing data
df = df[df['UPT']>0]

In [21]:
# set the indices
df['ID'] = df['MNAME'] + '-' + df['Mode']
df=df.set_index(['ID','Year'])

In [22]:
df['BUS_FLAG'] = np.where(df['Mode']=='Bus', 1, 0)
df['RAIL_FLAG'] = np.where(df['Mode']=='Rail', 1, 0)

In [23]:
# keep only the numeric columns -- the estimation will give an error otherwise
df = df.select_dtypes(include=[np.number])

In [24]:
# create a log of all fields
for col in df.columns:
    df[col+'_log'] = np.log(df[col]+1)
    

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: RuntimeWarning: invalid value encountered in log
  app.launch_new_instance()


In [25]:
# check the data
df.head()

CBSA  NEW_CBSA     UPT     VRM   UPT_ADJ  \
ID                         Year                                              
Abilene, TX Metro Area-Bus 2002  10180       NaN  493073  389678  493073.0   
                           2003  10180       NaN  475747  383239  475747.0   
                           2004  10180       NaN  469981  388907  469981.0   
                           2005  10180       NaN  507165  444176  507165.0   
                           2006  10180       NaN  493638  441300  493638.0   

                                  VRM_ADJ  VRH_ADJ  FARE_TOTAL  FARE_per_UPT  \
ID                         Year                                                
Abilene, TX Metro Area-Bus 2002  389678.0  29016.0    162609.0      0.329787   
                           2003  383239.0  28597.0    173563.0      0.364822   
                           2004  388907.0  29126.0    185046.0      0.393731   
                           2005  444176.0  31147.0    213965.0      0.421884   
                           2006  441300.0  30750.0    201795.0      0.408791   

                                 AREALANDPT      Tot_Pop  Tot_Instate_Pop  \
ID                         Year                                             
Abilene, TX Metro Area-Bus 2002   141756054  117258.2188      87723.96875   
                           2003   141756054  127098.1250      93677.12500   
                           2004   141756054  137113.5000      99787.50000   
                           2005   141756054  146427.0000     105269.00000   
                           2006   141756054  158548.0000     113266.00000   

                                 Tot_Outstate_Pop  Tot_NonUSA_POP  \
ID                         Year                                     
Abilene, TX Metro Area-Bus 2002       23752.17188     2987.484375   
                           2003       26964.31250     2834.062500   
                           2004       30227.75000     2648.750000   
                           2005       33286.00000     2591.000000   
                           2006       37165.00000     2023.000000   

                                 Total_Median_Income_Individual  \
ID                         Year                                   
Abilene, TX Metro Area-Bus 2002                     19756.67188   
                           2003                     19941.31250   
                           2004                     20132.75000   
                           2005                     20297.00000   
                           2006                     20570.00000   

                                 Native_Instate_Med_Inc_Indiv  \
ID                         Year                                 
Abilene, TX Metro Area-Bus 2002                   22848.10938   
                           2003                   21972.56250   
                           2004                   21085.75000   
                           2005                   20244.00000   
                           2006                   19222.00000   

                                 Native_Outstate_Med_Inc_Indiv  \
ID                         Year                                  
Abilene, TX Metro Area-Bus 2002                    17091.51563   
                           2003                    18472.93750   
                           2004                    19808.25000   
                           2005                    21328.00000   
                           2006                    22110.00000   

                                 Total_Pop_Poverty  Pop_Below100_Poverty  \
ID                         Year                                            
Abilene, TX Metro Area-Bus 2002          147631.94                 22.30   
                           2003          147198.25                 20.72   
                           2004          146801.00                 19.13   
                           2005          146258.00                 17.60   
                           2006          146298.00       

In [26]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + GasPrice_log \
                    + PCT_HH_NO_VEH \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4809
Estimator:                   PanelOLS   R-squared (Between):              0.9733
No. Observations:                5171   R-squared (Within):               0.4809
Date:                Mon, Oct 28 2019   R-squared (Overall):              0.9715
Time:                        11:13:47   Log-likelihood                   -1297.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      741.74
Entities:                         361   P-value                           0.0000
Avg Obs:                       14.324   Distribution:                  F(6,4804)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             741.74
                            

In [27]:
# include population + employment
df['POP_EMP'] = df['Tot_Pop'] + df['TOT_EMP_MSA']
df['HH_EMP'] = df['Total_HH'] + df['TOT_EMP_MSA']

df['POP_EMP_log'] = np.log(1+df['POP_EMP'])
df['HH_EMP_log'] = np.log(1+df['HH_EMP'])


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: RuntimeWarning: invalid value encountered in log
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: RuntimeWarning: invalid value encountered in log


In [28]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_log \
                    + POP_EMP_log \
                    + GasPrice_log \
                    + PCT_HH_NO_VEH \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4829
Estimator:                   PanelOLS   R-squared (Between):              0.8943
No. Observations:                5172   R-squared (Within):               0.4829
Date:                Mon, Oct 28 2019   R-squared (Overall):              0.8906
Time:                        11:13:49   Log-likelihood                   -1287.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      897.67
Entities:                         361   P-value                           0.0000
Avg Obs:                       14.327   Distribution:                  F(5,4806)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             897.67
                            

In [29]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_log \
                    + HH_EMP_log \
                    + GasPrice_log \
                    + PCT_HH_NO_VEH \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4808
Estimator:                   PanelOLS   R-squared (Between):              0.9340
No. Observations:                5173   R-squared (Within):               0.4808
Date:                Mon, Oct 28 2019   R-squared (Overall):              0.9310
Time:                        11:13:50   Log-likelihood                   -1298.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      890.16
Entities:                         361   P-value                           0.0000
Avg Obs:                       14.330   Distribution:                  F(5,4807)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             890.16
                            

In [30]:
# Use pop + emp

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_log \
                    + POP_EMP_log \
                    + GasPrice_log \
                    + PCT_HH_NO_VEH \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4829
Estimator:                   PanelOLS   R-squared (Between):              0.8943
No. Observations:                5172   R-squared (Within):               0.4829
Date:                Mon, Oct 28 2019   R-squared (Overall):              0.8906
Time:                        11:13:51   Log-likelihood                   -1287.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      897.67
Entities:                         361   P-value                           0.0000
Avg Obs:                       14.327   Distribution:                  F(5,4806)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             897.67
                            

In [31]:
#####################################################
# look at TNC effects

In [32]:
# test TNC effect

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_log \
                    + POP_EMP_log \
                    + GasPrice_log \
                    + PCT_HH_NO_VEH \
                    + TNC_FLAG \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4847
Estimator:                   PanelOLS   R-squared (Between):              0.8424
No. Observations:                5172   R-squared (Within):               0.4847
Date:                Mon, Oct 28 2019   R-squared (Overall):              0.8384
Time:                        11:13:52   Log-likelihood                   -1278.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      753.24
Entities:                         361   P-value                           0.0000
Avg Obs:                       14.327   Distribution:                  F(6,4805)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             753.24
                            

In [33]:
# test TNC effect

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_log \
                    + POP_EMP_log \
                    + GasPrice_log \
                    + PCT_HH_NO_VEH \
                    + YEARS_SINCE_TNC \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4830
Estimator:                   PanelOLS   R-squared (Between):              0.8876
No. Observations:                5172   R-squared (Within):               0.4830
Date:                Mon, Oct 28 2019   R-squared (Overall):              0.8838
Time:                        11:13:53   Log-likelihood                   -1287.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      748.03
Entities:                         361   P-value                           0.0000
Avg Obs:                       14.327   Distribution:                  F(6,4805)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             748.03
                            

In [34]:
# split bus vs rail TNC
df['YEARS_SINCE_TNC_BUS'] = df['BUS_FLAG'] * df['YEARS_SINCE_TNC']
df['YEARS_SINCE_TNC_RAIL'] = df['RAIL_FLAG'] * df['YEARS_SINCE_TNC']


In [35]:
# test TNC effect

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_log \
                    + POP_EMP_log \
                    + GasPrice_log \
                    + PCT_HH_NO_VEH \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4888
Estimator:                   PanelOLS   R-squared (Between):              0.8940
No. Observations:                5172   R-squared (Within):               0.4888
Date:                Mon, Oct 28 2019   R-squared (Overall):              0.8902
Time:                        11:14:58   Log-likelihood                   -1257.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      656.27
Entities:                         361   P-value                           0.0000
Avg Obs:                       14.327   Distribution:                  F(7,4804)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             656.27
                            

In [36]:
# Start from this base model

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_log \
                    + POP_EMP_log \
                    + GasPrice_log \
                    + PCT_HH_NO_VEH \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4829
Estimator:                   PanelOLS   R-squared (Between):              0.8943
No. Observations:                5172   R-squared (Within):               0.4829
Date:                Mon, Oct 28 2019   R-squared (Overall):              0.8906
Time:                        11:16:05   Log-likelihood                   -1287.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      897.67
Entities:                         361   P-value                           0.0000
Avg Obs:                       14.327   Distribution:                  F(5,4806)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             897.67
                            

In [37]:
# This is a linear model

mod=PanelOLS.from_formula('UPT_ADJ \
                    ~ VRM_ADJ \
                    + FARE_per_UPT \
                    + POP_EMP \
                    + GasPrice \
                    + PCT_HH_NO_VEH \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:                UPT_ADJ   R-squared:                        0.4022
Estimator:                   PanelOLS   R-squared (Between):              0.8357
No. Observations:                5174   R-squared (Within):               0.4022
Date:                Mon, Oct 28 2019   R-squared (Overall):              0.8296
Time:                        11:17:06   Log-likelihood                -9.342e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      646.97
Entities:                         361   P-value                           0.0000
Avg Obs:                       14.332   Distribution:                  F(5,4808)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             646.97
                            